In [6]:
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from glob import glob
from spreadsurfer.price_engine import FeatureEngineer
from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler

## chosen models
# min: models/2023-02-03T22:09:25-min_quantile0.5-score-73.0.cat
# max: models/2023-02-03T22:14:13-max_quantile0.7-score--154.0.cat


# cat_filename = 'models/2023-02-03T22:09:25-max_quantile0.5-score-77.0.cat'
# cat_filename = 'models/2023-02-03T22:14:13-max_quantile0.7-score--154.0.cat'

# cat_filename = 'models/2023-02-03T18:06:57-max_mape-score--21.0.cat'

# cat_filename = 'models/2023-02-03T21:57:57-min_huber0.4-score-108.0.cat'
# cat_filename = 'models/2023-02-03T21:34:07-min_poisson-score-191.0.cat'
# cat_filename = 'models/2023-02-03T21:50:04-min_quantile0.3-score--135.0.cat'
# cat_filename = '2023-02-03T20:13:00-min_rmse-score--10.0.cat'
# cat_filename = 'models/2023-02-01T09:14:50-mape-score-161.0.cat'

def load_and_score_mae(cat_filename, min=False):
    model = CatBoostRegressor()
    model.load_model(fname=cat_filename)
    print(f'catboost model loaded from {cat_filename}')

    pipeline = Pipeline(steps=[
        ('preprocessor', FeatureEngineer()),
        ('model', model)
    ])

    with open("neverseen.log","r") as f:
        input_from_log = ''.join([x for x in f.readlines() if 'collected' in x][-100:])

    mae = 0
    count = 0
    for line in input_from_log.split('\n'):
        if not line: continue
        count += 1
        json = '{' + line.split('{')[1]
        json = json.replace('nan', '0')
        sample = pd.DataFrame([eval(json)])
        if sample.wave_direction[0] not in ['min', 'max']: continue

        real = sample.last_price_delta_since_stabilized[0]
        if abs(real) < 0.2: continue
        sample.drop('last_price_delta_since_stabilized', axis=1, inplace=True)

        sample.loc[sample['wave_direction'] == 'min', 'wave_direction'] = 1
        sample.loc[sample['wave_direction'] == 'max', 'wave_direction'] = -1
        sample = sample.astype({"wave_direction": 'float64'})

        # if sample['wave_direction'][0] != (1 if min else -1):
        #     continue

        print('expected: ', real)
        guess = model.predict(sample)[0]
        guess *= 3.5
        print('predict: ', guess)
        mae += abs(real - guess)
        print()
    display(f'MAE: {mae / count}')
    return mae / count

# load_and_score_mae(cat_filename)

## search for best model
# models = [x for x in glob('models/*max_quantile*.cat')]
models = [x for x in glob('models/*max_quantile0.7-score--173*') + glob('models/quant0.5-over0.6-score160.cat')]
display(models)
results = {}
for i in models:
    try:
        mae = load_and_score_mae(i, min=False)
        results[mae] = i
    except Exception:
        print('skip with error')

scored = dict(sorted(results.items()))
print(scored)


['models/2023-02-07T13:50:42-max_quantile0.7-score--173.0.cat',
 'models/quant0.5-over0.6-score160.cat']

catboost model loaded from models/2023-02-07T13:50:42-max_quantile0.7-score--173.0.cat
expected:  -0.25
predict:  -0.07869720587047556

expected:  0.64
predict:  -0.37562149579231907

expected:  0.55
predict:  -0.2092530296802294

expected:  -0.5
predict:  -0.02374552028068292

expected:  -0.23
predict:  -0.06946174850837075

expected:  -1.59
predict:  -0.5585513024029966

expected:  1.14
predict:  -1.0100964363218061

expected:  0.94
predict:  -0.33473118278198477

expected:  -0.54
predict:  -1.0007408317403206

expected:  -0.21
predict:  -0.5318545346230258

expected:  -0.33
predict:  -0.32529638792026155

expected:  0.21
predict:  -1.8636543831806798

expected:  -0.43
predict:  -0.29666648074556606

expected:  -1.01
predict:  -0.10494002835252704

expected:  -0.2
predict:  -0.35675835823937346

expected:  3.18
predict:  -1.1981504674725976

expected:  -0.3
predict:  -0.29687648439914033

expected:  0.92
predict:  -1.363020736962059

expected:  0.27
predict:  -0.31488092965450376

ex

'MAE: 0.24537602658347596'

catboost model loaded from models/quant0.5-over0.6-score160.cat
expected:  -0.25
predict:  -0.41452340102996454

expected:  0.64
predict:  0.3733226507712276

expected:  0.55
predict:  0.5339683423777463

expected:  -0.5
predict:  -0.40410405437749247

expected:  -0.23
predict:  -0.4487327134906641

expected:  -1.59
predict:  -0.49466621751273865

expected:  1.14
predict:  1.5405338954621692

expected:  0.94
predict:  0.7522525991921426

expected:  -0.54
predict:  -0.62101851161895

expected:  -0.21
predict:  -0.5123089893465484

expected:  -0.33
predict:  -0.13947403445089415

expected:  0.21
predict:  1.1520194041408476

expected:  -0.43
predict:  -0.5791767543228119

expected:  -1.01
predict:  -0.4333368044346318

expected:  -0.2
predict:  -0.885220896257551

expected:  3.18
predict:  -0.24554186147508866

expected:  -0.3
predict:  -0.3488238630779705

expected:  0.92
predict:  0.7157942255451177

expected:  0.27
predict:  0.006083045258889006

expected:  1.73
predict:  0.3364732538

'MAE: 0.14209479467936784'

{0.14209479467936784: 'models/quant0.5-over0.6-score160.cat', 0.24537602658347596: 'models/2023-02-07T13:50:42-max_quantile0.7-score--173.0.cat'}
